# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw
Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw
Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw
Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw
Processing...
Done!


C:\Users\thinkpad\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Then, we define the model, object function and optimizer that we use to classify.

In [14]:
import torch.nn.functional as F
import torch.optim as optim
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super().__init__()
        #1*1*28*28
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(20 * 10 * 10, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        in_size = x.size(0)
        out= self.conv1(x) # 1* 10 * 24 *24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2) # 1* 10 * 12 * 12
        out = self.conv2(out) # 1* 20 * 10 * 10
        out = F.relu(out)
        out = out.view(in_size, -1) # 1 * 2000
        out = self.fc1(out) # 1 * 500
        out = F.relu(out)
        out = self.fc2(out) # 1 * 10
        out = F.log_softmax(out, dim = 1)
        return out
class Simplenet1(nn.Module):

    def __init__(self):

        super(Simplenet1, self).__init__()
        #self.flag = flag



        #self.layer_1 = nn.Linear(self.emb_dim, self.num_classes)
        self.firstpart = nn.Sequential(
          nn.Conv2d(1,10,5),
          nn.ReLU(),
          nn.MaxPool2d(2, 2),
          nn.Conv2d(10, 20, 3),
          nn.ReLU()
        )
        self.secondpart=nn.Sequential(
          nn.Linear(2000, 500),
          nn.ReLU(),
          nn.Linear(500, 10)



        )

    def forward(self, x):
                            # x shape os [batch_size, emb_dim]
        out = self.firstpart(x) # shape is [batch_size , 512]
        out=out.view(x.size(0), -1)
        out=self.secondpart(out)
        out = F.log_softmax(out, dim = 1)


        return out




    
model = Simplenet1()

# TODO:define loss function and optimiter
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [15]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    model.train()
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        output = model(images)
        loss =criterion(output, labels)
        loss.backward()
        optimizer.step()
    #print(epoch,  loss.item())
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
def test(model,test_loader):
    model.eval()
    test_loss =0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction = 'sum') # 将一批的损失相加
            pred = output.max(1, keepdim = True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    return( correct / len(test_loader.dataset))


test_accuracy = test(model, test_loader)
train_accuracy =test(model, train_loader)
print('Training accuracy: %0.2f%%' % (train_accuracy*100))
print('Testing accuracy: %0.2f%%' % (test_accuracy*100))
    
    
    
    



100%|██████████| 468/468 [00:27<00:00, 16.84it/s]

100%|██████████| 468/468 [00:29<00:00, 15.84it/s]

100%|██████████| 468/468 [00:32<00:00, 14.39it/s]

100%|██████████| 468/468 [00:35<00:00, 13.09it/s]

100%|██████████| 468/468 [00:39<00:00, 11.91it/s]

100%|██████████| 468/468 [00:36<00:00, 12.94it/s]

100%|██████████| 468/468 [00:35<00:00, 13.08it/s]

100%|██████████| 468/468 [00:35<00:00, 13.07it/s]

100%|██████████| 468/468 [00:36<00:00, 12.78it/s]

100%|██████████| 468/468 [00:40<00:00, 11.51it/s]


Training accuracy: 99.63%
Testing accuracy: 98.78%


#### Q5:
Please print the training and testing accuracy.